## Storm surge and wave climate
Notebook environment to migrate netcdf files to CF compliant zarr

In [1]:
# Optional; code formatter, installed as jupyter lab extension
#%load_ext lab_black
# Optional; code formatter, installed as jupyter notebook extension
%load_ext nb_black

<IPython.core.display.Javascript object>

### Set up modules and paths

In [2]:
# Import standard packages
import os
import pathlib
import sys
import numpy as np
import geopandas as gpd
import pandas as pd
import matplotlib.pyplot as plt
import xarray as xr
import netCDF4 as nc
import numpy.ma as ma

# Import custom functionality
from coclicodata.drive_config import p_drive
from coclicodata.etl.cf_compliancy_checker import check_compliancy, save_compliancy
from coastmonitor.io.utils import name_block

# Make root directories importable by appending root to path
cwd = pathlib.Path().resolve()
sys.path.append(os.path.dirname(cwd))

# Get root paths
home = pathlib.Path().home()
root = home.root

# Define (local and) remote drives
coclico_data_dir = p_drive.joinpath("11207608-coclico", "FULLTRACK_DATA")

# Workaround to the Windows OS (10) udunits error after installation of cfchecker: https://github.com/SciTools/iris/issues/404
os.environ["UDUNITS2_XML_PATH"] = str(
    home.joinpath(  # change to the udunits2.xml file dir in your Python installation
        r"Anaconda3\pkgs\udunits2-2.2.28-h892ecd3_0\Library\share\udunits\udunits2.xml"
    )
)

C:\Users\soest\AppData\Local\Temp\ipykernel_2600\86448705.py:6: DeprecationWarning: Shapely 2.0 is installed, but because PyGEOS is also installed, GeoPandas still uses PyGEOS by default. However, starting with version 0.14, the default will switch to Shapely. To force to use Shapely 2.0 now, you can either uninstall PyGEOS or set the environment variable USE_PYGEOS=0. You can do this before starting the Python process, or in your code before importing geopandas:

import os
os.environ['USE_PYGEOS'] = '0'
import geopandas

In the next release, GeoPandas will switch to using Shapely by default, even if PyGEOS is installed. If you only have PyGEOS installed to get speed-ups, this switch should be smooth. However, if you are using PyGEOS directly (calling PyGEOS functions on geometries from GeoPandas), this will then stop working and you are encouraged to migrate from PyGEOS to Shapely 2.0 (https://shapely.readthedocs.io/en/latest/migration_pygeos.html).
  import geopandas as gpd
c:\Users\

<IPython.core.display.Javascript object>

In [3]:
# Project paths & files (manual input)
WP_dir = coclico_data_dir.joinpath("WP3")
data_dir = WP_dir.joinpath("data")
ds_dir = data_dir.joinpath("NetCDF")
ds_filename = ds_dir.joinpath("CTP_MarineClimatologies.nc")
ds_out_filename = ds_dir.joinpath("CTP_MarineClimatologies")

CF_dir = data_dir.joinpath(r"CF")

<IPython.core.display.Javascript object>

In [4]:
ds = xr.open_dataset(ds_filename)
# Add CRS
ds.attrs["crs"] = 3035
ds

<xarray.Dataset> Size: 2MB
Dimensions:      (locs: 51010)
Coordinates:
    lon          (locs) float64 408kB ...
    lat          (locs) float64 408kB ...
Dimensions without coordinates: locs
Data variables:
    Hsmean       (locs) float64 408kB ...
    SSp99        (locs) float64 408kB ...
    tidal_range  (locs) float64 408kB ...
Attributes:
    title:         EU Coastal Target Points - Coastal marine climatologies
    summary:       Location of Coastal Target Points and their respective mea...
    institution:   Universidad de Cantabria, Instituto de Hidráulica Ambienta...
    keywords:      Waves, Storm surge, tide, coast, EU scale
    date_created:  28nd of February 2024
    project:       CoCliCo - Coastal Climate Core Services
    Conventions:   CF-1.8
    crs:           3035

<IPython.core.display.Javascript object>

In [7]:
# Change locs to stations
ds.rename_dims({'locs': 'stations'})

<xarray.Dataset> Size: 2MB
Dimensions:      (stations: 51010)
Coordinates:
    lon          (stations) float64 408kB ...
    lat          (stations) float64 408kB ...
Dimensions without coordinates: stations
Data variables:
    Hsmean       (stations) float64 408kB ...
    SSp99        (stations) float64 408kB ...
    tidal_range  (stations) float64 408kB ...
Attributes:
    title:         EU Coastal Target Points - Coastal marine climatologies
    summary:       Location of Coastal Target Points and their respective mea...
    institution:   Universidad de Cantabria, Instituto de Hidráulica Ambienta...
    keywords:      Waves, Storm surge, tide, coast, EU scale
    date_created:  28nd of February 2024
    project:       CoCliCo - Coastal Climate Core Services
    Conventions:   CF-1.8
    crs:           3035

<IPython.core.display.Javascript object>

In [8]:
%%capture cap --no-stderr
# check original CF compliancy

check_compliancy(testfile=ds_filename, working_dir=CF_dir)

<IPython.core.display.Javascript object>

In [9]:
# save original CF compliancy
save_compliancy(cap, testfile=ds_filename, working_dir=CF_dir)

# NOTE: File is already CF-compliant :)

{'P:\\11207608-coclico\\FULLTRACK_DATA\\WP3\\data\\NetCDF\\CTP_MarineClimatologies.nc': {'warnings': '0', 'errors': '0'}}


<IPython.core.display.Javascript object>

In [11]:
# export to zarr in write mode (to overwrite if exists)
ds.to_zarr(ds_dir.joinpath("%s.zarr" % ds_out_filename), mode="w")

<IPython.core.display.Javascript object>